# House Prices Regression

## Imports

In [64]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import random
import json

In [32]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [6]:
data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE'] = boston.target 
data.head()

CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  PRICE  
0     15.3  396.90   4.98   24.0  
1     17.8  396.90   9.14   21.6  
2     17.8  392.83   4.03   34.7  
3     18.7  394.63   2.94   33.4  
4     18.7  396.90   5.33   36.2

## Data Key

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University.

The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
prices and the demand for clean air', J. Environ. Economics & Management,
vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980.   N.B. Various transformations are used in the table on
pages 244-261 of the latter.

The Boston house-price data has been used in many machine learning papers that address regression
problems.   
     
.. topic:: References

   - Belsley, Kuh & Welsch, 'Regression diagnostics: Identifying Influential Data and Sources of Collinearity', Wiley, 1980. 244-261.
   - Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning. In Proceedings on the Tenth International Conference of Machine Learning, 236-243, University of Massachusetts, Amherst. Morgan Kaufmann.

In [13]:
data.describe()

CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO           B  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
std     28.148861    2.105710    8.707259  168.537116    2.164946   91.294864   
min      2.900000    1.129600    1.000000  187.000000   12.600000    0.320000   
25%     45.025000    2.100175    4.000000  279.000000   17.400000  375.377500   
50%     77.500000    3.207450    5.000000  330.000000   19.050000  391.440000   
75%     94.075000    5.188425   24.000000  666.000000   20.200000  396.225000   
max    100.000000   12.126500   24.000000  711.000000   22.000000  396.900000   

            LSTAT       PRICE  
count  506.000000  506.000000  
mean    12.653063   22.532806  
std      7.141062    9.197104  
min      1.730000    5.000000  
25%      6.950000   17.025000  
50%     11.360000   21.200000  
75%     16.955000   25.000000  
max     37.970000   50.000000

In [14]:
data.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64

In [15]:
corr = data.corr()
corr.shape

(14, 14)

In [16]:
plt.figure(figsize=(20,20))
sns.heatmap(corr, cbar=True, square= True,
            fmt='.1f', annot=True, 
            annot_kws={'size':15}, cmap='Reds')

In [18]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [19]:
#SPlitting into traing test dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [27]:
train_dataset = pd.concat((X_train, y_train), axis = 1)
test_dataset = pd.concat((X_test, y_test), axis = 1)

In [29]:
# os.mkdir("datasets/")

In [30]:
# train_dataset.to_csv("datasets/train.csv", index = False)
# test_dataset.to_csv("datasets/test.csv", index = False)

In [33]:
#Using XgBoost for our base model

reg = XGBRegressor()

In [34]:
reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

## Train Data

In [35]:
# Model prediction on train data
y_pred = reg.predict(X_train)

In [36]:
# Model Evaluation
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

R^2: 0.9999980912169968
Adjusted R^2: 0.9999980182341173
MAE: 0.008653071775274813
MSE: 0.00014367568030020436
RMSE: 0.011986479061851497


In [37]:
# Visualizing the differences between actual prices and predicted values
plt.scatter(y_train, y_pred)
plt.xlabel("Prices")
plt.ylabel("Predicted prices")
plt.title("Prices vs Predicted prices")
plt.show()

In [38]:
# Checking residuals
plt.scatter(y_pred,y_train-y_pred)
plt.title("Predicted vs residuals")
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.show()

## Test data

In [39]:
#Predicting Test data with the model
y_test_pred = reg.predict(X_test)


In [40]:
# Model Evaluation
acc_xgb = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_xgb)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

R^2: 0.8579952058455785
Adjusted R^2: 0.8446179426281331
MAE: 2.5309582816927056
MSE: 14.828150869974769
RMSE: 3.8507338092855457


In [55]:
# Importances
plt.figure(figsize=(20,20))
plt.bar(x = X_train.columns, height = reg.feature_importances_)

<Container object of 13 artists>

## Saving the Model

In [42]:
os.mkdir('models/')

In [43]:
reg.save_model('models/xgb_regressor.json')

In [49]:
with open('models/xgb_regressor_snapshot.bin', 'wb') as fd:
    pickle.dump(reg, fd)

## Testing the predictor

In [50]:
#Loading the model
with open('models/xgb_regressor_snapshot.bin', 'rb') as file:
    xgb_model = pickle.load(file)

## Generating test JSONs

In [73]:
test_dataset = test_dataset.reset_index(drop = True)

In [71]:
test_dataset = test_dataset.drop(columns=['index'])

In [74]:
test_dataset

CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS   RAD    TAX  \
0     0.21124  12.5   7.87   0.0  0.524  5.631  100.0  6.0821   5.0  311.0   
1     0.04297  52.5   5.32   0.0  0.405  6.565   22.9  7.3172   6.0  293.0   
2     0.13554  12.5   6.07   0.0  0.409  5.594   36.8  6.4980   4.0  345.0   
3     0.37578   0.0  10.59   1.0  0.489  5.404   88.6  3.6650   4.0  277.0   
4     0.38214   0.0   6.20   0.0  0.504  8.040   86.5  3.2157   8.0  307.0   
5     0.08826   0.0  10.81   0.0  0.413  6.417    6.6  5.2873   4.0  305.0   
6     0.01311  90.0   1.22   0.0  0.403  7.249   21.9  8.6966   5.0  226.0   
7     4.34879   0.0  18.10   0.0  0.580  6.167   84.0  3.0334  24.0  666.0   
8    14.43830   0.0  18.10   0.0  0.597  6.852  100.0  1.4655  24.0  666.0   
9     1.41385   0.0  19.58   1.0  0.871  6.129   96.0  1.7494   5.0  403.0   
10    0.03049  55.0   3.78   0.0  0.484  6.874   28.1  6.4654   5.0  370.0   
11    0.11460  20.0   6.96   0.0  0.464  6.538   58.7  3.9175   3.0  223.0   
12    2.63548   0.0   9.90   0.0  0.544  4.973   37.8  2.5194   4.0  304.0   
13    1.27346   0.0  19.58   1.0  0.605  6.250   92.6  1.7984   5.0  403.0   
14    0.47547   0.0   9.90   0.0  0.544  6.113   58.8  4.0019   4.0  304.0   
15    9.51363   0.0  18.10   0.0  0.713  6.728   94.1  2.4961  24.0  666.0   
16    0.11747  12.5   7.87   0.0  0.524  6.009   82.9  6.2267   5.0  311.0   
17    9.91655   0.0  18.10   0.0  0.693  5.852   77.8  1.5004  24.0  666.0   
18   28.65580   0.0  18.10   0.0  0.597  5.155  100.0  1.5894  24.0  666.0   
19    0.84054   0.0   8.14   0.0  0.538  5.599   85.7  4.4546   4.0  307.0   
20   73.53410   0.0  18.10   0.0  0.679  5.957  100.0  1.8026  24.0  666.0   
21    2.14918   0.0  19.58   0.0  0.871  5.709   98.5  1.6232   5.0  403.0   
22    0.09849   0.0  25.65   0.0  0.581  5.879   95.8  2.0063   2.0  188.0   
23    0.12802   0.0   8.56   0.0  0.520  6.474   97.1  2.4329   5.0  384.0   
24    0.03113   0.0   4.39   0.0  0.442  6.014   48.5  8.0136   3.0  352.0   
25    0.21161   0.0   8.56   0.0  0.520  6.137   87.4  2.7147   5.0  384.0   
26    0.43571   0.0  10.59   1.0  0.489  5.344  100.0  3.8750   4.0  277.0   
27    0.88125   0.0  21.89   0.0  0.624  5.637   94.7  1.9799   4.0  437.0   
28    6.44405   0.0  18.10   0.0  0.584  6.425   74.8  2.2004  24.0  666.0   
29    0.17120   0.0   8.56   0.0  0.520  5.836   91.9  2.2110   5.0  384.0   
..        ...   ...    ...   ...    ...    ...    ...     ...   ...    ...   
122   0.08244  30.0   4.93   0.0  0.428  6.481   18.5  6.1899   6.0  300.0   
123   0.09266  34.0   6.09   0.0  0.433  6.495   18.4  5.4917   7.0  329.0   
124   0.28392   0.0   7.38   0.0  0.493  5.708   74.3  4.7211   5.0  287.0   
125   6.80117   0.0  18.10   0.0  0.713  6.081   84.4  2.7175  24.0  666.0   
126   2.01019   0.0  19.58   0.0  0.605  7.929   96.2  2.0459   5.0  403.0   
127   7.05042   0.0  18.10   0.0  0.614  6.103   85.1  2.0218  24.0  666.0   
128   1.15172   0.0   8.14   0.0  0.538  5.701   95.0  3.7872   4.0  307.0   
129  13.35980   0.0  18.10   0.0  0.693  5.887   94.7  1.7821  24.0  666.0   
130   0.52693   0.0   6.20   0.0  0.504  8.725   83.0  2.8944   8.0  307.0   
131   0.75026   0.0   8.14   0.0  0.538  5.924   94.1  4.3996   4.0  307.0   
132   0.61154  20.0   3.97   0.0  0.647  8.704   86.9  1.8010   5.0  264.0   
133   0.06664   0.0   4.05   0.0  0.510  6.546   33.1  3.1323   5.0  296.0   
134   0.36894  22.0   5.86   0.0  0.431  8.259    8.4  8.9067   7.0  330.0   
135   0.17446   0.0  10.59   1.0  0.489  5.960   92.1  3.8771   4.0  277.0   
136  10.67180   0.0  18.10   0.0  0.740  6.459   94.8  1.9879  24.0  666.0   
137   0.03659  25.0   4.86   0.0  0.426  6.302   32.2  5.4007   4.0  281.0   
138   0.26169   0.0   9.90   0.0  0.544  6.023   90.4  2.8340   4.0  304.0   
139   0.17783   0.0   9.69   0.0  0.585  5.569   73.5  2.3999   6.0  391.0   
140   3.32105   0.0  19.58   1.0  0.871  5.403  100.0  1.3216   5.0  403.0   
141  15.28800   0.0  18.10   0.

In [75]:
dict(test_dataset.iloc[10])

{'AGE': 28.1,
 'B': 387.97,
 'CHAS': 0.0,
 'CRIM': 0.03049,
 'DIS': 6.4654,
 'INDUS': 3.78,
 'LSTAT': 4.61,
 'NOX': 0.484,
 'PRICE': 31.2,
 'PTRATIO': 17.6,
 'RAD': 5.0,
 'RM': 6.874,
 'TAX': 370.0,
 'ZN': 55.0}

In [63]:
os.mkdir('test')

In [110]:
X_test

CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS   RAD    TAX  \
8     0.21124  12.5   7.87   0.0  0.524  5.631  100.0  6.0821   5.0  311.0   
289   0.04297  52.5   5.32   0.0  0.405  6.565   22.9  7.3172   6.0  293.0   
68    0.13554  12.5   6.07   0.0  0.409  5.594   36.8  6.4980   4.0  345.0   
211   0.37578   0.0  10.59   1.0  0.489  5.404   88.6  3.6650   4.0  277.0   
226   0.38214   0.0   6.20   0.0  0.504  8.040   86.5  3.2157   8.0  307.0   
70    0.08826   0.0  10.81   0.0  0.413  6.417    6.6  5.2873   4.0  305.0   
55    0.01311  90.0   1.22   0.0  0.403  7.249   21.9  8.6966   5.0  226.0   
470   4.34879   0.0  18.10   0.0  0.580  6.167   84.0  3.0334  24.0  666.0   
409  14.43830   0.0  18.10   0.0  0.597  6.852  100.0  1.4655  24.0  666.0   
154   1.41385   0.0  19.58   1.0  0.871  6.129   96.0  1.7494   5.0  403.0   
344   0.03049  55.0   3.78   0.0  0.484  6.874   28.1  6.4654   5.0  370.0   
272   0.11460  20.0   6.96   0.0  0.464  6.538   58.7  3.9175   3.0  223.0   
310   2.63548   0.0   9.90   0.0  0.544  4.973   37.8  2.5194   4.0  304.0   
160   1.27346   0.0  19.58   1.0  0.605  6.250   92.6  1.7984   5.0  403.0   
319   0.47547   0.0   9.90   0.0  0.544  6.113   58.8  4.0019   4.0  304.0   
454   9.51363   0.0  18.10   0.0  0.713  6.728   94.1  2.4961  24.0  666.0   
11    0.11747  12.5   7.87   0.0  0.524  6.009   82.9  6.2267   5.0  311.0   
399   9.91655   0.0  18.10   0.0  0.693  5.852   77.8  1.5004  24.0  666.0   
413  28.65580   0.0  18.10   0.0  0.597  5.155  100.0  1.5894  24.0  666.0   
25    0.84054   0.0   8.14   0.0  0.538  5.599   85.7  4.4546   4.0  307.0   
418  73.53410   0.0  18.10   0.0  0.679  5.957  100.0  1.8026  24.0  666.0   
153   2.14918   0.0  19.58   0.0  0.871  5.709   98.5  1.6232   5.0  403.0   
124   0.09849   0.0  25.65   0.0  0.581  5.879   95.8  2.0063   2.0  188.0   
108   0.12802   0.0   8.56   0.0  0.520  6.474   97.1  2.4329   5.0  384.0   
345   0.03113   0.0   4.39   0.0  0.442  6.014   48.5  8.0136   3.0  352.0   
103   0.21161   0.0   8.56   0.0  0.520  6.137   87.4  2.7147   5.0  384.0   
209   0.43571   0.0  10.59   1.0  0.489  5.344  100.0  3.8750   4.0  277.0   
129   0.88125   0.0  21.89   0.0  0.624  5.637   94.7  1.9799   4.0  437.0   
432   6.44405   0.0  18.10   0.0  0.584  6.425   74.8  2.2004  24.0  666.0   
106   0.17120   0.0   8.56   0.0  0.520  5.836   91.9  2.2110   5.0  384.0   
..        ...   ...    ...   ...    ...    ...    ...     ...   ...    ...   
238   0.08244  30.0   4.93   0.0  0.428  6.481   18.5  6.1899   6.0  300.0   
302   0.09266  34.0   6.09   0.0  0.433  6.495   18.4  5.4917   7.0  329.0   
323   0.28392   0.0   7.38   0.0  0.493  5.708   74.3  4.7211   5.0  287.0   
459   6.80117   0.0  18.10   0.0  0.713  6.081   84.4  2.7175  24.0  666.0   
166   2.01019   0.0  19.58   0.0  0.605  7.929   96.2  2.0459   5.0  403.0   
423   7.05042   0.0  18.10   0.0  0.614  6.103   85.1  2.0218  24.0  666.0   
33    1.15172   0.0   8.14   0.0  0.538  5.701   95.0  3.7872   4.0  307.0   
394  13.35980   0.0  18.10   0.0  0.693  5.887   94.7  1.7821  24.0  666.0   
225   0.52693   0.0   6.20   0.0  0.504  8.725   83.0  2.8944   8.0  307.0   
24    0.75026   0.0   8.14   0.0  0.538  5.924   94.1  4.3996   4.0  307.0   
257   0.61154  20.0   3.97   0.0  0.647  8.704   86.9  1.8010   5.0  264.0   
175   0.06664   0.0   4.05   0.0  0.510  6.546   33.1  3.1323   5.0  296.0   
253   0.36894  22.0   5.86   0.0  0.431  8.259    8.4  8.9067   7.0  330.0   
210   0.17446   0.0  10.59   1.0  0.489  5.960   92.1  3.8771   4.0  277.0   
445  10.67180   0.0  18.10   0.0  0.740  6.459   94.8  1.9879  24.0  666.0   
82    0.03659  25.0   4.86   0.0  0.426  6.302   32.2  5.4007   4.0  281.0   
312   0.26169   0.0   9.90   0.0  0.544  6.023   90.4  2.8340   4.0  304.0   
499   0.17783   0.0   9.69   0.0  0.585  5.569   73.5  2.3999   6.0  391.0   
142   3.32105   0.0  19.58   1.0  0.871  5.403  100.0  1.3216   5.0  403.0   
376  15.28800   0.0  18.10   0.

In [76]:
for idx in range(10):
    index = random.randint(0, 150)
    sample = dict(test_dataset.iloc[index])
    with open(f'test/data_sample_{idx}.json', 'w') as file:
        json.dump(sample, file)

## Test Local Prediction

In [120]:
import requests

In [167]:
with open('test/data_sample_0.json', 'r') as file:
    data = json.load(file)

In [168]:
res = requests.post(url= 'http://0.0.0.0:5000/predict',
               json = data)

In [169]:
res.json()

{'PRICE': 15.38182258605957}